In [1]:
import sys
# append the path of the parent directory
sys.path.append("..")

In [11]:
import math
import os
import time


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as patches

import seaborn as sns
import time
import json
import pandas as pd
from ctypes import c_int32
from itertools import product

from scipy.stats import pearsonr
from importlib import reload

from lib import sketches, visualization_utils, encoders, ploting, pacha_sketch_new
reload(ploting)
reload(sketches)
reload(visualization_utils)
reload(encoders)

reload(pacha_sketch_new)

from lib.sketches import BloomFilter, CountMinSketch, H3HashFunctions, HashFunctionFamily
from lib.visualization_utils import visualize_badic_cover, plot_b_adic_cubes
from lib.encoders import minimal_b_adic_cover, minimal_spatial_b_adic_cover, BAdicCube, BAdicRange
from lib.pacha_sketch_new import PachaSketch, ADTree, BFParameters, CMParameters, build_pacha_sketch_uniform_size, \
    CategoricalPredicate, NumericalPredicate, build_pacha_sketch_from_json_file

from lib.ploting import set_style, plot_ylabel, plot_legend

# set_style()

In [3]:
ad_tree = ADTree()
ad_tree.add_dimension({"m","f","d"})
ad_tree.add_dimension({"Baden-Württemberg", "Bavaria", "Berlin", "Brandenburg", "Bremen", 
        "Hamburg", "Hesse", "Lower Saxony", "Mecklenburg-Vorpommern", 
        "North Rhine-Westphalia", "Rhineland-Palatinate", "Saarland", 
        "Saxony", "Saxony-Anhalt", "Schleswig-Holstein", "Thuringia"
    })
ad_tree.add_dimension({chr(i) for i in range(97, 123)})

In [23]:
input_data = pd.read_parquet("data/paper_example_200k.parquet")

In [12]:
pacha_sketch = build_pacha_sketch_uniform_size(
    levels=10,
    num_dimensions=6,
    cat_col_map=[1,0,2],
    num_col_map=[3,4,5],
    bases=[2,2,2],
    ad_tree=ad_tree,
    cm_params=CMParameters(delta=0.01, eps=0.00005),
    bf_params=BFParameters(p=0.0001, n_values=200000),
    cat_index_parameters=BFParameters(p=0.0001, n_values=4*200000),
    num_index_parameters=BFParameters(p=0.0001, n_values=10*200000),
    n_sparse_levels=2)

In [15]:
from pympler import asizeof
print(asizeof.asizeof(pacha_sketch)/1024/1024, "MB")

75.12442779541016 MB


In [5]:
pacha_sketch.save_to_file("sketches/test.json.gz")

In [13]:
pacha_sketch2 = build_pacha_sketch_from_json_file("sketches/test.json.gz")

In [15]:
pacha_sketch == pacha_sketch2

True

In [29]:
pacha_sketch.update_data_frame(input_data, workers=os.cpu_count())

Updating: 100%|██████████| 200000/200000 [02:26<00:00, 1366.81it/s]


In [119]:
pacha_sketch.query([
    CategoricalPredicate({"*"}),
    CategoricalPredicate({"m"}),
    CategoricalPredicate({"*"}),
    NumericalPredicate(trivial=True),
    NumericalPredicate(0, 100),
    NumericalPredicate(30, 40)
    ])

np.int64(49)

In [83]:
BFParameters(p=0.0001, n_values=10*200000).build_sketch().size

38340234

In [79]:
int(np.ceil(math.e/0.00005))

54366

In [67]:
cube2 = BAdicCube([BAdicRange(2,1,0), BAdicRange(2,1,0)])
hash(cube2)

9002410152673085047

In [63]:
tuple( r for r in range(6) )

(0, 1, 2, 3, 4, 5)

In [30]:
n = None
n > 0

TypeError: '>' not supported between instances of 'NoneType' and 'int'